## Create Keycloak resources

Wait until Keycloak is running

In [1]:
!docker inspect --format='json' my-keycloak | yq '.[0].State'

{
  "Status": "running",
  "Running": true,
  "Paused": false,
  "Restarting": false,
  "OOMKilled": false,
  "Dead": false,
  "Pid": 8917,
  "ExitCode": 0,
  "Error": "",
  "StartedAt": "2024-08-08T15:07:40.162311318Z",
  "FinishedAt": "0001-01-01T00:00:00Z"
}


Then create a sample realm and client with some roles and users matching the test environment.

In [11]:
import requests
import json
from dotenv import set_key

OIDC_SERVER_URL = "http://0.0.0.0:9999"
ADMIN_USERNAME = "admin"
ADMIN_PASSWORD = "admin"

access_token: str = ""

In [3]:
def get_token():
    token_url = f"{OIDC_SERVER_URL}/realms/master/protocol/openid-connect/token"

    token_data = {
        "grant_type": "password",
        "client_id": "admin-cli",
        "username": ADMIN_USERNAME,
        "password": ADMIN_PASSWORD,
    }

    token_response = requests.post(token_url, data=token_data)
    if token_response.status_code == 200:
        global access_token
        access_token = token_response.json()["access_token"]
        return access_token
    else:
        print(
            f"Failed to obtain access token: {token_response.status_code} - {token_response.text}"
        )
        raise Exception("Not authenticated")


def keycloak_post(endpoint, data=None):
    url = f"{OIDC_SERVER_URL}/admin/{endpoint}"
    print(f"Creating {endpoint}")
    global access_token
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {access_token}",
    }
    response = requests.request("POST", url, headers=headers, data=json.dumps(data))
    print(f"POST response.status_code is {response.status_code}")
    return response.status_code


def keycloak_get(endpoint):
    url = f"{OIDC_SERVER_URL}/admin/{endpoint}"
    global access_token
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {access_token}",
    }
    response = requests.request("GET", url, headers=headers)
    print(f"GET response.status_code is {response.status_code}")
    return response.json()


def create_realm(realm_name):
    data = {"realm": realm_name, "enabled": "true"}
    keycloak_post("realms", data=data)
    response = keycloak_get(f"realms/{realm_name}")
    return response["id"]


def create_client(realm_name, client_name):
    data = {
        "clientId": client_name,
        "enabled": "true",
        "redirectUris": [
            "http://localhost:8000/*",
            "http://127.0.0.1:8000/*",
            "http://0.0.0.0:8000/*",
        ],
        "publicClient": False,
        "authorizationServicesEnabled": True,
        "protocol": "openid-connect",
        "standardFlowEnabled": True,
        "directAccessGrantsEnabled": True,
        "serviceAccountsEnabled": True,
    }
    keycloak_post(f"realms/{realm_name}/clients", data=data)
    response = keycloak_get(f"realms/{realm_name}/clients")
    client = None
    for c in response:
        if c["clientId"] == client_name:
            client = c
            break
    client_id = client["id"]
    client_secret = client["secret"]
    return client_id, client_secret


def create_client_roles(realm_name, client_id, roles):
    for role_name in roles:
        data = {"name": role_name, "clientRole": True}
        keycloak_post(f"realms/{realm_name}/clients/{client_id}/roles", data=data)

    response = keycloak_get(f"realms/{realm_name}/clients/{client_id}/roles")
    roles_by_name = dict((role["name"], role["id"]) for role in response)
    print(roles_by_name)
    return roles_by_name


def create_user_with_roles(
    realm_name, username, password, client_id, roles_by_name, roles
):
    data = {
        "username": username,
        "enabled": True,
        "email": f"{username}@poc.com",
        "emailVerified": True,
        "firstName": "user",
        "lastName": f"{username}",
        "credentials": [{"type": "password", "value": password}],
        "realmRoles": [],
    }
    keycloak_post(f"realms/{realm_name}/users", data=data)
    response = keycloak_get(f"realms/{realm_name}/users")
    user = None
    for u in response:
        if u["username"] == username:
            user = u
            break
    user_id = user["id"]

    data = [
        {
            "id": roles_by_name[role_name],
            "name": role_name,
        }
        for role_name in roles
    ]
    keycloak_post(
        f"realms/{realm_name}/users/{user_id}/role-mappings/clients/{client_id}",
        data=data,
    )

In [4]:
get_token()

'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICIweUNoR2N3ZG1ZNXRSNkJ5MGtVQjRmZDVPZTVqM2JLak5pX2hKUmc2ejdFIn0.eyJleHAiOjE3MjMxMzAyNTgsImlhdCI6MTcyMzEzMDE5OCwianRpIjoiNGNkOWRmMTQtZTYzYy00YTFiLTg1ODQtMGE4OGZkYTFiOGI2IiwiaXNzIjoiaHR0cDovLzAuMC4wLjA6OTk5OS9yZWFsbXMvbWFzdGVyIiwic3ViIjoiOTAyM2FjMTEtZWI0Yi00YjlkLTk2MDItNWJlMWRkMGU0NzRjIiwidHlwIjoiQmVhcmVyIiwiYXpwIjoiYWRtaW4tY2xpIiwic2Vzc2lvbl9zdGF0ZSI6ImNhYzg5YjAxLTMxOTYtNDMzZi1iMGJiLTBmZGY3MDUyNmZmNiIsImFjciI6IjEiLCJzY29wZSI6InByb2ZpbGUgZW1haWwiLCJzaWQiOiJjYWM4OWIwMS0zMTk2LTQzM2YtYjBiYi0wZmRmNzA1MjZmZjYiLCJlbWFpbF92ZXJpZmllZCI6ZmFsc2UsInByZWZlcnJlZF91c2VybmFtZSI6ImFkbWluIn0.xrLurTh9eiU4Kyqk6EGER5qaf9MPDJFT34xkUMYbl2ZmzUz8VnKSsJtFlkPSSgMhAprecjlPWLrKqRdAK35YKnXBFUBxEznCTa6fykdHh_3tuTbwt3vtLRxAogTDghnVsRL3VK295jnclUgQzaJUIoFGXgapfxihfn6em6PqBaYWEcKlbd0lhNA_BmAe-XFOnz2o6kUd0TGKCD43plhhL0QfpHmiJ509rqGkuywTgo5WTXFP52Zc6v9b3eReSNXefyuMtZA9gsyCxzhFl1azZh8CPe6UuadcTd1O0d4mbIRiIu2aUrjlplkFrB1KMSGSBD1MaS88PibetMth6P5xgg'

In [5]:
realm_name = "rbac_example"
client_name = "app"
password = "password"

In [6]:
realm_id = create_realm(realm_name)
client_id, client_secret = create_client(realm_name, client_name)

roles_by_name = create_client_roles(
    realm_name, client_id, ["reader", "fresh_writer", "store_admin", "batch_admin"]
)

Creating realms
POST response.status_code is 201
GET response.status_code is 200
Creating realms/rbac_example/clients
POST response.status_code is 201
GET response.status_code is 200
Creating realms/rbac_example/clients/1fcdfed3-9b28-421f-883d-cde6b000c26d/roles
POST response.status_code is 201
Creating realms/rbac_example/clients/1fcdfed3-9b28-421f-883d-cde6b000c26d/roles
POST response.status_code is 201
Creating realms/rbac_example/clients/1fcdfed3-9b28-421f-883d-cde6b000c26d/roles
POST response.status_code is 201
Creating realms/rbac_example/clients/1fcdfed3-9b28-421f-883d-cde6b000c26d/roles
POST response.status_code is 201
GET response.status_code is 200
{'fresh_writer': '1d2cdaa5-efd7-471f-9b10-b68de2f11cd1', 'reader': '856f4a68-687a-4319-9441-588a6c18fb13', 'batch_admin': '42b414ab-4e43-4b3e-83a0-011393f2952a', 'store_admin': '9ad09a29-0493-47f5-83ad-5de081025c04', 'uma_protection': '0752be64-28bb-430e-8d3c-38ad0c8b1d1b'}


In [7]:
create_user_with_roles(
    realm_name, "reader", password, client_id, roles_by_name, ["reader"]
)
create_user_with_roles(
    realm_name,
    "writer",
    password,
    client_id,
    roles_by_name,
    ["fresh_writer"],
)
create_user_with_roles(
    realm_name,
    "batch_admin",
    password,
    client_id,
    roles_by_name,
    ["batch_admin"],
)
create_user_with_roles(
    realm_name,
    "admin",
    password,
    client_id,
    roles_by_name,
    ["store_admin"],
)

Creating realms/rbac_example/users
POST response.status_code is 201
GET response.status_code is 200
Creating realms/rbac_example/users/345ce240-c1d6-4df7-8f86-f6717a24ae19/role-mappings/clients/1fcdfed3-9b28-421f-883d-cde6b000c26d
POST response.status_code is 204
Creating realms/rbac_example/users
POST response.status_code is 201
GET response.status_code is 200
Creating realms/rbac_example/users/34a9dd95-8504-41f4-89f5-e243a9581f55/role-mappings/clients/1fcdfed3-9b28-421f-883d-cde6b000c26d
POST response.status_code is 204
Creating realms/rbac_example/users
POST response.status_code is 201
GET response.status_code is 200
Creating realms/rbac_example/users/2b57245d-7cdf-4fef-b455-b1f1551c0bf3/role-mappings/clients/1fcdfed3-9b28-421f-883d-cde6b000c26d
POST response.status_code is 204
Creating realms/rbac_example/users
POST response.status_code is 201
GET response.status_code is 200
Creating realms/rbac_example/users/c01fd547-036f-4efa-896c-9e8f3cf1f31e/role-mappings/clients/1fcdfed3-9b28-

In [8]:
print(f"Realm {realm_name} setup completed.")
print(
    f"Client {client_name} created with ID {client_id} and secret {client_secret}"
)

env_file = ".env"
with open(env_file, "w") as file:
    pass

# Write property P=1 to the .env file
set_key(env_file, "OIDC_SERVER_URL", OIDC_SERVER_URL)
set_key(env_file, "REALM", realm_name)
set_key(env_file, "CLIENT_ID", client_name)
set_key(env_file, "CLIENT_SECRET", client_secret)
set_key(env_file, "PASSWORD", password)
print(f"Settings configured in {env_file}")

Realm rbac_example setup completed.
Client app created with ID 1fcdfed3-9b28-421f-883d-cde6b000c26d and secret eFGTp6ZJ0tARKYPbdMKUZ1J8PJNWfXR8
Settings configured in .env


The [.env](.env) file contains the settings of the created realm, including the client secret to be used to connect the server.

In [12]:
!cat .env

OIDC_SERVER_URL='http://0.0.0.0:9999'
REALM='rbac_example'
CLIENT_ID='app'
CLIENT_SECRET='eFGTp6ZJ0tARKYPbdMKUZ1J8PJNWfXR8'
PASSWORD='password'
